# SITCOM-[761](https://jira.lsstcorp.org/browse/SITCOM-761) - Create plots to analyze the Vibration Monitoring System (VMS) data
This contains the following: 
 1. a description of the VMS data
 2. how to obtain the VMS data
 3. example plots of VMS data (telemetry, psd, spectrogram)

Currently this notebook does not include:
 * Transfer function between the VMS and accelerometers

## Vibration Monitoring System (VMS)
The VMS is a set of accelerometers mounted on the M1M3, M2, and Camera Rotator that measure high frequency accelerations (not including gravity). The data rate is 200 hz, and so can be used to identify vibrations up to 100 hz. These instruments and their calibration are NIST certified. More information can be found [confluence](https://confluence.lsstcorp.org/x/jQhUCQ)

There are muliple sensors on each of the three components, that show up as different channels. 
 * M1M3 has 3 sensors
 * M2 has 6 sensors
 * Camera Rotator has 3 sensors


## Accessing data from the VMS
VMS data is saved daily into files on vms-data.cp.lsst.org (hopefully these files will be transfered to the usdf but for now individual hdf files on the machine vms-data.cp.lsst.org at `/vms-data/yyyy/mm/vms_name-yyyy-mm-ddThh:mm.hdf`, where the `vms_name` can be M1M3, M2, or Rotator.

The function in this notebook `vms_data_to_frame` can be used to read these hdf files

This notebook uses an example vms file located on the usdf at `/sdf/group/rubin/shared/mtm1m3_test_files/M1M3-2023-09-07T00:00.hdf`

### important note VMS data taken before September 2023 seems to have a bug in the timestamps and should not be used
In the future this notebook will be updated with data taken to slews and a comparison to efd telemetry 

In [ ]:
import sys, time, os, asyncio
import scipy.stats as stats
from scipy.signal import find_peaks
from scipy import signal
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient, clipDataToEvent, calcNextDay
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [ ]:
# Functions to get data
key_m1m3_dict={'1 X': 'm1m3_x_1', 
            '1 Y': 'm1m3_y_1', 
            '1 Z': 'm1m3_z_1', 
            '2 X': 'm1m3_x_2', 
            '2 Y': 'm1m3_z_2', # note these two have been 
            '2 Z': 'm1m3_y_2', # switched pending SUMMIT-7911
            '3 X': 'm1m3_x_3', 
            '3 Y': 'm1m3_y_3', 
            '3 Z': 'm1m3_z_3'
            }
key_m2_dict={'1 X': 'm2_x_1', 
            '1 Y': 'm2_y_1', 
            '1 Z': 'm2_z_1', 
            '2 X': 'm2_x_2', 
            '2 Y': 'm2_z_2', 
            '2 Z': 'm2_y_2', 
            '3 X': 'm2_x_3', 
            '3 Y': 'm2_z_3', 
            '3 Z': 'm2_y_3', 
            '4 X': 'm2_x_4', 
            '4 Y': 'm2_y_4', 
            '4 Z': 'm2_z_4', 
            '5 X': 'm2_x_5', 
            '5 Y': 'm2_z_5', 
            '5 Z': 'm2_y_5',
            '6 X': 'm2_x_6', 
            '6 Y': 'm2_z_6', 
            '6 Z': 'm2_y_6', 
            }
def vms_data_to_frame(filename, vms_type, begin_time=None, end_time=None):
    """
    Converts VMS data in the given HDF5 file to a Pandas DataFrame.

    Args:
    filename: Path to the HDF5 file containing the VMS data.
    vms_type: The type of VMS data in the file. Must be "m1m3", "m2", or
      "rotator".
    begin_time: The start time of the data to include in the DataFrame. If None,
      all data will be included.
    end_time: The end time of the data to include in the DataFrame. If None, all
      data will be included.

    Returns:
    A Pandas DataFrame containing the VMS data.
    """
    if vms_type == "m1m3":
        key_dict=key_m1m3_dict
    elif vms_type=="m2":
        key_dict=key_m2_dict
    elif vms_type=="rotator":
        raise NotImplementedError
    else:
        raise ValueError('vms_type must be m1m3,m2, or rotator')

    f = h5py.File(filename, 'r')
    times = f['timestamp'][::1]
    dkeys = 'XYZ'
   
    data_dict = {}
    if (begin_time is not None) & (end_time is not None): 
        sel = (times > begin_time) & (times < end_time)
    else: 
        sel = np.ones(times.size).astype(bool)
    data_dict['times'] = times[sel]  
    for key in key_dict.keys():
        data_dict[key_dict[key]] = f[key][::1][sel]
    data_frame = pd.DataFrame(data_dict)
    for j in np.arange(int(len(key_dict)/3)) +1:
        data_frame[f"total_{j}"] = np.linalg.norm(
            data_frame[[f"{vms_type}_{i}_{j}" for i in ["x","y","z"]]].values, axis=1
        )
    
    
    return data_frame

def get_efd_data(begin, end, client):

    """Extract all the MTMount data from the EFD and add to dict.

    Args:
        begin (str): The start time of the query.
        end (str): The end time of the query.
        client (object): influx client

    Returns:
        dict: A dictionary containing the MTMount data.
    """

    query_dict = {}

    query_dict["el"] = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=["private_sndStamp", "private_efdStamp", "actualPosition", "actualVelocity", "actualTorque"],
        begin=begin,
        end=end,
        prePadding=0,
        postPadding=0,
        warn=False,
    )
    query_dict["az"] = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=["private_sndStamp", "private_efdStamp", "actualPosition", "actualVelocity", "actualTorque"],
        begin=begin,
        end=end,
        prePadding=0,
        postPadding=0,
        warn=False,
    )
    return query_dict


In [ ]:
def get_freq_psd(vals, timestep):

    """
    Calculates the frequency power spectrum of a signal.

    Args:
        vals (np.array): The signal values.
        timestep (float): The time step between samples.

    Returns:
        tuple: The frequencies and power spectral density.
    """

    # Remove the mean from the signal.

    meanval = np.mean(vals)
    signal = vals - meanval

    # Calculate the length of the signal.

    N = len(signal)

    # Calculate the power spectral density.

    psd = np.abs(np.fft.rfft(np.array(signal) * 1)) ** 2

    # Calculate the frequencies.

    frequencies = np.fft.rfftfreq(N, timestep)

    return (frequencies, psd)

def get_peak_points(freq, psd, height=0.01):
    """
    Get the peak points of the power spectral density (PSD).

    Args:
        freq (numpy.ndarray): The frequency vector.
        psd (numpy.ndarray): The power spectral density.
        height (float): The minimum peak height.

    Returns:
        numpy.ndarray: The peak points.
    """

    # Find the peak indices and heights.
    peak_ind, peak_dict = find_peaks(psd, height=height)
    peaks = freq[peak_ind]

    # If there are no peaks, return None.
    if len(peaks) < 1:
        return None

    # Find the sub-peaks within each group of peaks that are close in frequency.
    points = []
    for i, peak in enumerate(peaks):
        sel = (abs(peaks - peak) < 1)
        sub_peaks = peaks[sel]
        sub_heights = peak_dict['peak_heights'][sel]
        points.append(sub_peaks[np.argmax(sub_heights)])

    # Return the unique peak points.
    return np.unique(np.array(points))

### Loading VMS data
The `vms_data_to_frame` function loads the vms data for the desired timestamp and also computes the total acceleration in each channel 

We leave in an efd query, that can be used in the future for vms data that overlaps with movements of the TMA

In [ ]:
vms_m1m3_filename="/sdf/group/rubin/shared/mtm1m3_test_files/M1M3-2023-09-07T00:00.hdf"

begin_time=Time('2023-09-07 00:13:30', format="iso", scale="utc")
end_time=Time('2023-09-07 00:15:30', format="iso", scale="utc")

In [ ]:
client=makeEfdClient()
efd_dict=get_efd_data(begin_time, end_time, client)

vms_m1m3_data=vms_data_to_frame(vms_m1m3_filename, vms_type="m1m3",begin_time=begin_time.unix, end_time=end_time.unix)

### Plotting the VMS data
 1. Plot of total acceleration telemetry (with an offset) for each of the 3 m1m3 vms channels
 2. A Power Spectral Distribution (psd) of each axis (xyz) of each channel of the m1m3 vms data
 3. Example spectrogram of the total acceleration for each channel 

In [ ]:
fig, ax=plt.subplots(1, dpi=125, sharex=True, figsize=(10,3))
plt.suptitle(f"M1M3 VMS\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}\n", y=0.99)

for i in np.arange(3):

    ax.scatter(Time(vms_m1m3_data["times"], format="unix").datetime, 
               vms_m1m3_data[f"total_{i+1}"]  - 0.001 * i,
               label=f"total channel {i+1} \noffset = {- 0.001 * i:0.3f}", s=1)
ax.set(ylabel="total acceleration", ylim=(-0.0025, 0.0015))
ax.legend(ncol=3, loc=9, edgecolor="white")


In [ ]:
plt.figure(dpi=175,figsize=(10,5))
plt.suptitle(f"M1M3 VMS\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}\n PSD for all axes of all channels", y=0.99)
step=0.1
for i,key in enumerate([i for i in sorted(key_m1m3_dict.values()) if ("m1m3" in i)]):
    
    freq,psd=get_freq_psd(vms_m1m3_data[key], np.mean(np.diff(vms_m1m3_data["times"])))
    points_x=get_peak_points(freq, psd)
    if points_x is not None:
        points_y=np.ones_like(points_x) * i * step * -1
        print(f"{key}: {str([int(i) for i in points_x])[1:-1]} Hz")
    else: 
        print(f"{key} no peaks")
    plt.plot(freq,psd - i * step, label=key, zorder=9, lw=1)

plt.legend(ncol=3, edgecolor="k", facecolor="white", loc=9, framealpha=1)
plt.ylim(-0.05 - i * step, 0.35)
plt.xlabel("frequency [Hz]")
plt.ylabel("psd + offset")
plt.grid(visible=True, axis="x", ls="dashed", alpha=0.4)
_=plt.xticks(np.arange(0,110,10))



In [ ]:
fs=1/np.mean(np.diff(vms_m1m3_data["times"]))
fig,axs=plt.subplots(3, dpi=175, figsize=(10,4), sharex=True)
plt.suptitle(f"M1M3\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}", y=0.97)


ax=axs[0]
ax.specgram(vms_m1m3_data["total_1"] - np.mean(vms_m1m3_data["total_1"]), Fs=fs, detrend="mean", vmin=-110)
ax.set(ylabel="Channel 1\nFreq [hz]", yticks=np.arange(25,125,25))

ax=axs[1]
ax.specgram(vms_m1m3_data["total_2"] - np.mean(vms_m1m3_data["total_2"]), Fs=fs, detrend="mean", vmin=-110)
ax.set(ylabel="Channel 2\nFreq [hz]", yticks=np.arange(25,125,25))

ax=axs[2]
ax.specgram(vms_m1m3_data["total_3"] - np.mean(vms_m1m3_data["total_3"]), Fs=fs, detrend="mean", vmin=-110)
ax.set(ylabel="Channel 3\nFreq [hz]", yticks=np.arange(25,125,25))


plt.subplots_adjust(hspace=0.1)
